<a href="https://colab.research.google.com/github/rajan-raj-22/SOCIAL_MEDIA_POST_GENRATOR/blob/main/AI_Content_Genrator_Voice_Clone_4_%E2%9C%85%20(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# === Create music directory and upload background music ===
!mkdir -p music


In [2]:
# === Install Dependencies (Colab/Linux) ===
!apt-get update
!apt-get install -y imagemagick fontconfig ttf-mscorefonts-installer fonts-dejavu portaudio19-dev ffmpeg
!fc-cache -fv
!sed -i 's/rights=\"none\"/rights=\"read|write\"/g' /etc/ImageMagick*/policy.xml
!pip install TTS moviepy diffusers transformers accelerate scipy torchaudio gradio pydub

# === Imports ===
import os, gc, re, torch, requests, datetime, random
from transformers import CLIPTokenizer
from diffusers import StableDiffusionXLPipeline
from moviepy.editor import *
from pydub import AudioSegment
from TTS.api import TTS
import gradio as gr

GROQ_API_KEY = "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"



# === Load Tokenizer ===
clip_tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 128 kB in 1s (92.4 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Don

  if event.key is 'enter':



tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

In [3]:
# === Load Models ===
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = StableDiffusionXLPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16).to(device)
tts_model = TTS("tts_models/multilingual/multi-dataset/your_tts", progress_bar=False).to(device)

# === Memory Management ===
def free_cuda():
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    gc.collect()

# === Helpers ===
def clean_output(text):
    return text.encode("utf-16", "surrogatepass").decode("utf-16", "ignore")

def fallback_story(topic):
    title = f"Why {topic.capitalize()} Matters"
    story = f"{topic.capitalize()} is shaping the world. Let's explore how it's transforming our lives and future."
    return title, story

def clip_truncate(prompt, max_tokens=77):
    tokens = clip_tokenizer.tokenize(prompt)
    return clip_tokenizer.convert_tokens_to_string(tokens[:max_tokens])

def clean_text_for_tts(text):
    text = re.sub(r'\*\*+', '', text)
    text = re.sub(r'\[.*?\]\(.*?\)', '', text)
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'https?://\S+', '', text)
    text = text.replace("*", "")
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    merged = []
    buffer = ""
    for line in lines:
        if len(line.split()) < 5:
            buffer += " " + line
        else:
            if buffer:
                merged.append(buffer.strip())
                buffer = ""
            merged.append(line)
    if buffer:
        merged.append(buffer.strip())
    return " ".join(merged)

model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/737 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

...

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/10.3G [00:00<?, ?B/s]

vae_1_0/diffusion_pytorch_model.safetens(…):   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

 > Downloading model to /root/.local/share/tts/tts_models--multilingual--multi-dataset--your_tts
 > Model's license - CC BY-NC-ND 4.0
 > Check https://creativecommons.org/licenses/by-nc-nd/4.0/ for more info.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Model fully restored. 

In [4]:
def convert_to_wav_if_needed(file_path):
    try:
        ext = os.path.splitext(file_path)[1].lower()
        if ext != ".wav":
            audio = AudioSegment.from_file(file_path)
            wav_path = file_path.rsplit(".", 1)[0] + ".wav"
            audio.export(wav_path, format="wav")
            print(f"[🎵] Converted to WAV: {wav_path}")
            return wav_path
        return file_path
    except Exception as e:
        print(f"[⚠️] Failed to convert voice sample to WAV: {e}")
        return None

def generate_tts_coqui(text, out_path_wav, speaker_wav=None):
    print("[🔊] Generating TTS using Coqui...")
    try:
        cleaned = clean_text_for_tts(text[:2000])
        if speaker_wav:
            speaker_wav = convert_to_wav_if_needed(speaker_wav)
            if not speaker_wav or not os.path.isfile(speaker_wav):
                print("[⚠️] Skipping custom voice due to invalid or failed conversion.")
                speaker_wav = None
        tts_model.tts_to_file(text=cleaned, speaker_wav=speaker_wav, language="en", file_path=out_path_wav)
        print("[✅] TTS audio saved at:", out_path_wav)
    except Exception as e:
        raise RuntimeError(f"[❌] Coqui TTS generation failed: {e}")


In [5]:
def generate_story_llama3(topic, word_count=900):
    prompt = f"You are a skilled LinkedIn copywriter creating a social media post (~{word_count} words) on the topic: '{topic}'.\n\nRespond in the following format only:\nTitle: <insert title>\nStory: <insert story here>"
    try:
        res = requests.post(
            "https://api.groq.com/openai/v1/chat/completions",
            json={"model": "llama3-70b-8192", "messages": [{"role": "user", "content": prompt}]},
            headers={"Authorization": f"Bearer {GROQ_API_KEY}"}
        )
        output = res.json()['choices'][0]['message']['content']
        if "Title:" in output and "Story:" in output:
            title = output.split("Title:")[1].split("\n")[0].strip()
            story = output.split("Story:")[1].strip()
            os.makedirs("post", exist_ok=True)
            with open("post/story.txt", "w", encoding="utf-8") as f:
                f.write(f"Title: {title}\n\nStory: {story}")
            return title, story
        else:
            return fallback_story(topic)
    except Exception as e:
        print(f"[Groq Error] {e}")
        return fallback_story(topic)

In [6]:
def generate_image_prompt_from_topic(topic):
    prompt = f"""
You are an expert AI prompt engineer. Given the topic: '{topic}', create a highly detailed, visually rich prompt suitable for Stable Diffusion XL.

Ensure the prompt is vivid, descriptive, and highly specific with elements like lighting, background setting, mood, color tone, camera type, focus depth, and physical characteristics.
Use phrases like: ultra-detailed, cinematic, masterpiece, photorealistic, 8k, epic composition, trending on ArtStation.
Avoid generic or vague words. Output only the prompt.
"""
    try:
        res = requests.post(
            "https://api.groq.com/openai/v1/chat/completions",
            json={
                "model": "llama3-70b-8192",
                "messages": [{"role": "user", "content": prompt}]
            },
            headers={"Authorization": f"Bearer {GROQ_API_KEY}"}
        )
        return clip_tokenizer.convert_tokens_to_string(clip_tokenizer.tokenize(res.json()['choices'][0]['message']['content'])[:77])
    except:
        return clip_tokenizer.convert_tokens_to_string(clip_tokenizer.tokenize(f"photorealistic image of {topic}, ultra sharp, 8k, cinematic lighting")[:77])


def ken_burns_zoom_with_captions(image_path, duration, top_text="", zoom=1.1):
    background = ColorClip(size=(1080, 1920), color=(0, 0, 0), duration=duration)
    image_clip = ImageClip(image_path).resize(height=1440).set_duration(duration)

    # Use scale effect instead of cropping to zoom the image itself
    zoomed = image_clip.resize(lambda t: zoom ** (t / duration)).set_position((0, 240))

    caption = TextClip(top_text, fontsize=48, color="white", font="DejaVu-Serif-Bold",
                       method='caption', align='center', size=(1000, 200))
    caption = caption.set_position(("center", 40)).set_duration(duration)
    return CompositeVideoClip([background, zoomed, caption])

In [11]:
def process_pipeline(topic, output_type, word_count, voice_sample, bgm_audio, bgm_volume):
    yield "📝 Generating story...", None, None, None
    os.makedirs("post", exist_ok=True)
    title, story = generate_story_llama3(topic, word_count)

    prompt = generate_image_prompt_from_topic(topic)
    image_paths = []

    if output_type in ["Image", "Text Image Video", "Video"]:
        yield "🖼️ Generating images...", None, None, None
        for i in range(5):
            img = pipe(prompt=prompt, guidance_scale=8.0, generator=torch.manual_seed(i)).images[0]
            path = f"post/image_{i}.png"
            img.save(path)
            image_paths.append(path)

    out_path = None
    if output_type in ["Text Image Video", "Video"]:
        yield "🎙️ Generating voice-over...", None, None, None
        generate_tts_coqui(story, out_path_wav="post/audio.wav", speaker_wav=voice_sample)

        yield "🎬 Creating video clips...", None, None, None
        audio_clip = AudioFileClip("post/audio.wav")
        clips = [ken_burns_zoom_with_captions(img, duration=audio_clip.duration/5, top_text=title) for img in image_paths]
        final_video = concatenate_videoclips(clips)

        if bgm_audio:
            yield "🎵 Adding background music...", None, None, None
            bg_music = AudioFileClip(bgm_audio).volumex(bgm_volume)
            mixed_audio = CompositeAudioClip([audio_clip, bg_music.set_duration(audio_clip.duration)])
        else:
            mixed_audio = audio_clip

        final_clip = final_video.set_audio(mixed_audio)
        out_path = f"post/video_{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}.mp4"

        yield "📽️ Rendering video...", None, None, None
        final_clip.write_videofile(out_path, fps=24, codec="libx264", audio_codec="aac")

    yield "✅ Done!", f"Title: {title}\n\nStory: {story}", image_paths, out_path


In [ ]:
# === Custom CSS ===
custom_css = """
#title h2 {
  color: #ff4b4b !important;
  font-size: 32px !important;
  font-weight: 800;
}
.gr-button {
  background-color: #ff4b4b !important;
  color: white !important;
  font-weight: bold !important;
  border-radius: 8px;
  font-size: 20px !important;
}
.gr-button:hover {
  background-color: #e43e3e !important;
}
.gr-box, .gr-textbox, .gr-dropdown, .gr-image, .gr-accordion, .gr-file, .gr-slider, .gr-radio {
  background-color: rgba(0, 0, 0, 0.6) !important;
  border-radius: 12px;
  color: white !important;
}
html, body, .gradio-container {
  background-image: url('https://i.ibb.co/8WG4fZS/vibrant-colors-flow-abstract-wave-pattern-generated-by-ai.jpg');
  background-size: cover;
  background-position: center;
  background-attachment: fixed;
  font-family: 'Segoe UI', sans-serif;
  color: white;
  font-size: 18px !important;
}
label, input, select, textarea {
  font-size: 18px !important;
}
"""




# === Gradio UI ===
def run_gradio():
    with gr.Blocks(css=custom_css) as demo:
        gr.Markdown("## 🎬 AI Social Media Post Generator (Voice + Visual)", elem_id="title")
        with gr.Row():
            with gr.Column():
                topic = gr.Textbox(label="📌 Topic")
                output_type = gr.Radio(choices=["Text", "Image", "Video", "Text Image Video"], label="🎞️ Output Type")
                word_count = gr.Slider(500, 1500, value=900, step=100, label="✍️ Word Count")
                voice_sample = gr.File(label="🎤 Voice Sample (Optional)", type="filepath")
                bgm_audio = gr.File(label="🎵 BGM Music (Optional)", type="filepath")
                bgm_volume = gr.Slider(minimum=0.0, maximum=1.0, value=0.2, step=0.05, label="🔊 BGM Volume")
                generate_btn = gr.Button("🚀 Generate Post")
                status = gr.Textbox(label="⏱ Status", value="Idle", interactive=False)
            with gr.Column():
                image_preview = gr.Gallery(label="🖼️ Preview", columns=3, rows=2, object_fit="contain", height=400)
                result = gr.Textbox(label="📜 Output Summary", lines=14)
                download = gr.File(label="⬇️ Download Video")

        generate_btn.click(
    fn=process_pipeline,
    inputs=[topic, output_type, word_count, voice_sample, bgm_audio, bgm_volume],
    outputs=[status, result, image_preview, download]
)




    demo.launch(debug=True, share=True)

if __name__ == "__main__":
    run_gradio()


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4786deb2ee88d02e84.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['emerald eyes']
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['emerald eyes']


  0%|          | 0/50 [00:00<?, ?it/s]